In [ ]:
#loading necessary modules...
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision 
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import random
import torchtext.data as data
import csv
from tqdm import tqdm
from google.colab import drive
import pandas as pd
import string
import nltk
import torchtext
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
import re
import spacy
nlp = spacy.load('en')
import time
import sys, os

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#Mount Google Drive where files are housed.
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/My\ Drive/Colab Notebooks

/content/gdrive/My Drive/Colab Notebooks


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu") 

In [ ]:
#For randomization.
seed = 1997
torch.manual_seed(seed)

#Use 1 value to set up new process train/test files.
process_new_data = 1
#Use 1 value to train a new model. Set to 0 to load previously saved model.
train_new_data = 1

In [ ]:
#paths for the saved files. Set to a variable for reproducibility on other datasets.
train_dir = "train.csv"
process_train = "process_train.csv"
test_dir = "test.csv"
process_test = "process_test.csv"

In [ ]:
traindata = pd.read_csv(train_dir)
traindata.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
#Inspiration for this function gotten from https://www.kaggle.com/prabhatkumarsahu/toxic-comment-classification-thecaffeinedev
#Used to clean text data into form that can be input into vocabulary.
punctuations = string.punctuation
stopwords_list = stopwords.words("english")
spacy_tokenizer = torchtext.data.utils.get_tokenizer('spacy')
lemmatizer = WordNetLemmatizer()
def processing(text):
  
    def tokenizer(text):
        text = str.split(text)
        return text
    
    def remove_punctuations(sentence):
        result = "".join([i if i not in punctuations and not i.isdigit() else " " for i in sentence])
        return result
    
    def word_lemmatizer(sentence):
        result = lemmatizer.lemmatize(sentence)
        return result
    
    def word_lowercase(sentence):
        return sentence.lower()
    
    def remove_URL(text):
        url = re.compile(r'https?://\S+|www\.\S+')
        html = re.compile(r'<.*?>')
        text = html.sub(r'',text)
        text = url.sub(r'',str(text))
        return text
  
    def remove_newline(text):
        return text.rstrip("\n")
    
    def clean_comment(sentence):
        result = []
        sentence = remove_newline(sentence)
        sentence = remove_URL(sentence)
        sentence = word_lowercase(sentence)
        sentence = word_lemmatizer(sentence)
        sentence = remove_punctuations(sentence)
        sentence = tokenizer(sentence)

        result = " ".join(sentence)
        return result
     
    text = clean_comment(text)
    if text == "":
        text = "None"
    return text

In [ ]:
#new file where cleaned comments from test files will be written to.
if process_new_data:
    with open(test_dir, "r", encoding="utf8") as in_csv, open(process_test, "w", newline="", encoding="utf8") as out_csv:
        read = csv.reader(in_csv)
        write = csv.writer(out_csv)
        next(read, None) 
        for i in tqdm(read):
            i[1] = processing(i[1])
            try:
                write.writerow(i)
            except Exception as e:
               print(e)

153164it [00:14, 10248.39it/s]


In [ ]:
#new file where cleaned comments from train files will be written to.
if process_new_data:
    with open(train_dir, "r", encoding="utf8") as in_csv, open(process_train, "w", newline="", encoding="utf8") as out_csv:
        read = csv.reader(in_csv)
        write = csv.writer(out_csv)
        next(read, None) # Skip header
        for i in tqdm(read):
            i[1] = processing(i[1])
            try:
                write.writerow(i)
            except Exception as e:
               print(e)

159571it [00:15, 10449.32it/s]


In [ ]:
def token_words(sentence):
    tokens = str.split(sentence)
    return tokens

In [ ]:
#https://towardsdatascience.com/use-torchtext-to-load-nlp-datasets-part-i-5da6f1c89d84
TEXT = data.Field(batch_first = True,
                  tokenize = token_words,
                  stop_words=stopwords_list)
LABEL = data.LabelField(dtype = torch.float)
ID = data.LabelField()
ID2 = data.Field(sequential=False)

In [ ]:
#https://towardsdatascience.com/use-torchtext-to-load-nlp-datasets-part-i-5da6f1c89d84
features = [["id",ID], ["text", TEXT], ["toxic",LABEL],["s_toxic",LABEL],
          ["obscene",LABEL],["threat",LABEL],["insult",LABEL],["id_hate",LABEL]]
test_features = [["id",ID2], ["text", TEXT]]

data_train = data.TabularDataset(process_train,
                              format = "csv",
                              fields=features,
                              skip_header=True)

data_test = data.TabularDataset(process_test,
                              format= "csv",
                              fields=test_features,
                             skip_header=False)

In [ ]:
#Randomizing the datasets.
import random

data_train, data_val = data_train.split(split_ratio=0.8, random_state=random.seed(seed))

In [ ]:
#Building the corpus vocabulary.
vocab_size = 40000
TEXT.build_vocab(data_train,
                 min_freq = 3,
                 max_size = vocab_size, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(data_train)
ID.build_vocab(data_train)
ID2.build_vocab(data_test)

Unique words in training set: 40002
Unique labels in training set: 2


In [ ]:
#Set up for the Batch iterator.
BATCH_SIZE = 64
train_iterator, val_iterator = data.BucketIterator.splits((data_train, data_val),
                                                  batch_size=BATCH_SIZE,
                                                  device = device)
test_iterator = data.BucketIterator(data_test,
                                batch_size=BATCH_SIZE,
                                shuffle=False,
                                device = device)

In [ ]:
data_train

In [ ]:
#Set up of Convolutional Model used to solve problem.
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (i, embedding_dim)) 
                                    for i in filter_sizes
                                    ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        embedded = self.embedding(text)
        embedded = embedded.unsqueeze(1)
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        cat = self.dropout(torch.cat(pooled, dim = 1))
            
        return self.fc(cat)

In [ ]:
#Hyperparameters for model.
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 6
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-5,weight_decay=2e-5)
criterion = nn.BCEWithLogitsLoss().to(device)
print(model)

CNN(
  (embedding): Embedding(40002, 100, padding_idx=1)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(5, 100), stride=(1, 1))
  )
  (fc): Linear(in_features=300, out_features=6, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [ ]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [ ]:
#Function to reach into batch and for each label, return a (batch_size, 1) dimension tensor.
def get_labels(batch):
    toxic = batch.toxic.unsqueeze(1)
    s_toxic = batch.s_toxic.unsqueeze(1)
    obscene = batch.obscene.unsqueeze(1)
    threat = batch.threat.unsqueeze(1)
    insult = batch.insult.unsqueeze(1)
    id_hate = batch.id_hate.unsqueeze(1)
    labels = torch.cat((toxic,s_toxic,obscene,
                        threat,insult,id_hate),dim=1)
    return labels

In [ ]:
#training step by step function.
def train_pace(model, optimizer, criterion, batch):
    batch_size = len(batch)
    model.train()
    
    optimizer.zero_grad()
    text = batch.text.view(batch_size, -1)
    labels = get_labels(batch)

    outputs = model(text)
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()

    return loss.item()

In [ ]:
#Training function that runs if train_new_data is set to 1. 
if train_new_data:
    epochs = 2
    loss_list = []
    print("Training starting!")
    for epoch in range(epochs):
        for i, batch in enumerate(train_iterator):
            train_loss = train_pace(model,optimizer, criterion, batch)
            loss_list.append(train_loss)
            print(f"Epoch: [{epoch+1}/{epochs}] | Iterations: [{i+1}/{len(train_iterator)}] | Training loss: {train_loss:.3f}")
    torch.save(model.state_dict(), "model.pt")
          
        
    print("Training Done!")

Training starting!
Epoch: [1/2] | Iterations: [1/1995] | Training loss: 0.625
Epoch: [1/2] | Iterations: [2/1995] | Training loss: 0.631
Epoch: [1/2] | Iterations: [3/1995] | Training loss: 0.603
Epoch: [1/2] | Iterations: [4/1995] | Training loss: 0.621
Epoch: [1/2] | Iterations: [5/1995] | Training loss: 0.621
Epoch: [1/2] | Iterations: [6/1995] | Training loss: 0.608
Epoch: [1/2] | Iterations: [7/1995] | Training loss: 0.612
Epoch: [1/2] | Iterations: [8/1995] | Training loss: 0.612
Epoch: [1/2] | Iterations: [9/1995] | Training loss: 0.611
Epoch: [1/2] | Iterations: [10/1995] | Training loss: 0.602
Epoch: [1/2] | Iterations: [11/1995] | Training loss: 0.584
Epoch: [1/2] | Iterations: [12/1995] | Training loss: 0.594
Epoch: [1/2] | Iterations: [13/1995] | Training loss: 0.575
Epoch: [1/2] | Iterations: [14/1995] | Training loss: 0.562
Epoch: [1/2] | Iterations: [15/1995] | Training loss: 0.575
Epoch: [1/2] | Iterations: [16/1995] | Training loss: 0.584
Epoch: [1/2] | Iterations: [17

In [ ]:
#Change train_new_data value to 0, if you want to load previously saved model.
if not train_new_data:
    model.load_state_dict(torch.load("model.pt"))

In [ ]:
#Predicts toxicity level of all test case comments.
def predict_test_cases(model, test_iterator):
    result = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(test_iterator):
            batch_size = len(batch)
            text = batch.text.view(batch_size,-1).long()
            ids = batch.id.squeeze().cpu()
            output = model(text)
            output = torch.sigmoid(output).cpu()
            for i,j in zip(ids,output):
                result.append([ID2.vocab.itos[i.numpy()],j.numpy()])
    return result

In [ ]:
result = predict_test_cases(model, test_iterator)

100%|██████████| 2394/2394 [11:13<00:00,  3.55it/s]


# **Exploration**

In [ ]:
#Gotten from https://github.com/bentrevett/pytorch-sentiment-analysis/issues/40
#You feed in a text string and the function returns a tensor showing the toxicty level of the comments. Ranging from 0 - 1.
def sentiment_prediction(model, sentence, min_len = 5):
    model.eval()
    tokenized = [i.text for i in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[w] for w in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction

In [ ]:
label_list = ['toxic', 'severe toxic', 'obscene', 'threat', 'insult', 'identity hate']

In [ ]:
def output_results(tensor_results, label_list):

  print("Here are the results!")
  print("This comment is ", (tensor_results.tolist()[0][0])*100, "%", label_list[0],'.')
  print("This comment is ", (tensor_results.tolist()[0][1])*100, "%", label_list[1], '.')
  print("This comment is ", (tensor_results.tolist()[0][2])*100, "%", label_list[2], '.')
  print("This comment is ", (tensor_results.tolist()[0][3])*100, "%", label_list[3], '.')
  print("This comment is ", (tensor_results.tolist()[0][4])*100, "%", label_list[4], '.')
  print("This comment is ", (tensor_results.tolist()[0][5])*100, "%", label_list[5], '.')

In [ ]:
output_results(sentiment_prediction(model,text2), label_list)

Here are the results!
This comment is  97.2859799861908 % toxic .
This comment is  46.15640044212341 % severe toxic .
This comment is  93.64231824874878 % obscene .
This comment is  11.011741310358047 % threat .
This comment is  90.4722511768341 % insult .
This comment is  24.513758718967438 % identity hate .


In [ ]:
text2 = processing('stupid bitch')
output_results(sentiment_prediction(model,text2), label_list)

Here are the results!
This comment is  97.2859799861908 % toxic .
This comment is  46.15640044212341 % severe toxic .
This comment is  93.64231824874878 % obscene .
This comment is  11.011741310358047 % threat .
This comment is  90.4722511768341 % insult .
This comment is  24.513758718967438 % identity hate .


In [ ]:
text3 = processing('Hello how are you')
output_results(sentiment_prediction(model,text3), label_list)

Here are the results!
This comment is  36.656081676483154 % toxic .
This comment is  15.772013366222382 % severe toxic .
This comment is  30.28900921344757 % obscene .
This comment is  10.665873438119888 % threat .
This comment is  30.298712849617004 % insult .
This comment is  16.324447095394135 % identity hate .


In [ ]:
text4 = processing('Fuck Donald trump')
output_results(sentiment_prediction(model,text4), label_list)

Here are the results!
This comment is  77.93609499931335 % toxic .
This comment is  21.43966108560562 % severe toxic .
This comment is  67.56911277770996 % obscene .
This comment is  11.928174644708633 % threat .
This comment is  67.50600337982178 % insult .
This comment is  17.962075769901276 % identity hate .


In [ ]:
text5 = processing('i love you')
output_results(sentiment_prediction(model,text5), label_list)

Here are the results!
This comment is  23.88479709625244 % toxic .
This comment is  8.818437159061432 % severe toxic .
This comment is  14.886708557605743 % obscene .
This comment is  5.9532638639211655 % threat .
This comment is  16.656996309757233 % insult .
This comment is  10.665958374738693 % identity hate .
